# Stereo Vision: Bản đồ độ chênh lệch (disparity map), Tái tạo 3D và Hình học Epipolar
Notebook này minh họa cách tính toán bản đồ độ chênh lệch, tái tạo đám mây điểm 3D và ước lượng ma trận cơ bản để vẽ các đường epipolar sử dụng hình ảnh stereo.

## Bước 1: Nhập các thư viện cần thiết
Chúng ta sẽ sử dụng OpenCV để xử lý hình ảnh và NumPy để tính toán số học.

In [ ]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Bước 2: Tải hình ảnh Stereo
Tải hình ảnh bên trái và bên phải cho tầm nhìn stereo.

In [ ]:
# Load stereo images
left_image = cv2.imread('input/left.png', cv2.IMREAD_GRAYSCALE)
right_image = cv2.imread('input/right.png', cv2.IMREAD_GRAYSCALE)

# Display the images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image')
plt.axis('off')
plt.show()

## Bước 3: Tính toán Bản đồ Độ chênh lệch
Sử dụng Stereo Block Matching (BM) hoặc Stereo Global Block Matching (SGBM) để tính toán bản đồ độ chênh lệch.

In [ ]:
# Compute disparity map using StereoSGBM
stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=16*5,  # Must be divisible by 16
    blockSize=5,
    P1=8*3*5**2,
    P2=32*3*5**2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32
)

disparity = stereo.compute(left_image, right_image).astype(np.float32) / 16.0

# Display disparity map
plt.figure(figsize=(10, 5))
plt.imshow(disparity, cmap='plasma')
plt.colorbar()
plt.title('Disparity Map')
plt.axis('off')
plt.show()

## Bước 4: Tái tạo Đám mây Điểm 3D
Tái tạo đám mây điểm 3D sử dụng bản đồ độ chênh lệch và các tham số camera.

In [ ]:
# Define camera parameters
focal_length = 1.0  # Example focal length
baseline = 0.1  # Example baseline in meters

# Reconstruct 3D points
h, w = left_image.shape
Q = np.float32([[1, 0, 0, -w/2],
                [0, -1, 0, h/2],
                [0, 0, 0, -focal_length],
                [0, 0, 1/baseline, 0]])

points_3D = cv2.reprojectImageTo3D(disparity, Q)
mask = disparity > disparity.min()
points_3D = points_3D[mask]

# Visualize point cloud
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_3D[:, 0], points_3D[:, 1], points_3D[:, 2], s=0.1, c=points_3D[:, 2], cmap='plasma')
plt.title('3D Point Cloud')
plt.show()

## Bước 5: Ước lượng Ma trận Cơ bản và Vẽ các Đường Epipolar
Ước lượng ma trận cơ bản sử dụng các điểm tương ứng và vẽ các đường epipolar trên các hình ảnh đầu vào.

In [ ]:
# Detect keypoints and match features
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(left_image, None)
kp2, des2 = orb.detectAndCompute(right_image, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

# Extract matched points
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

# Estimate fundamental matrix
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# Draw epipolar lines
def draw_epipolar_lines(img1, img2, lines, pts1, pts2):
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [img1.shape[1], -(r[2] + r[0] * img1.shape[1]) / r[1]])
        img1_color = cv2.line(img1_color, (x0, y0), (x1, y1), color, 1)
        img1_color = cv2.circle(img1_color, tuple(pt1), 5, color, -1)
        img2_color = cv2.circle(img2_color, tuple(pt2), 5, color, -1)
    return img1_color, img2_color

lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F).reshape(-1, 3)
img1_epilines, img2_epilines = draw_epipolar_lines(left_image, right_image, lines1, pts1, pts2)

# Display images with epipolar lines
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(img1_epilines)
plt.title('Epipolar Lines on Left Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img2_epilines)
plt.title('Epipolar Lines on Right Image')
plt.axis('off')
plt.show()